<center>Progetto realizzato da Elena Curti (matr. 185431)

# Crimes Database
</center>

## Introduzione

## Requisiti


## Operazioni iniziali
Creare un database ...

In [1]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "password1234"))
print("Connessione al database eseguita correttamente!")

Connessione al database eseguita correttamente!


## Descrizione del database

## 1. Quali sono gli agenti di polizia più importanti?


### Cypher query

In [2]:
def print_n_results(cq, n=100):
    ris = graph.run(cq)
    ris.sample_size = n
    display(ris)

cq = """ 
MATCH (o:Officer)<-[:INVESTIGATED_BY]-(c:Crime) 
WITH o, count(c) as num_invest
RETURN o.badge_no as badge_number, o.name as nome, o.surname as cognome, num_invest
ORDER BY num_invest DESC
LIMIT 10 ;
"""
print_n_results(cq)


# TODO questo si può rimuovere, usarlo solo per capire come cambiano i risultati
ris1 = [i["badge_number"] + " " + i["nome"] +" " + i["cognome"] for i in graph.run(cq).data()] 
def check_results(cq):
    res = [i["badge_number"] + " " + i["nome"] +" " + i["cognome"] for i in graph.run(cq).data()] 
    return res == ris1



badge_number,nome,cognome,num_invest
48-0216838,Madelon,DeSousa,50
04-5664884,Cloe,Ings,47
58-9758255,Kania,Notti,46
27-2227814,Worthy,Nettles,45
86-7588000,Ricca,Miskimmon,44
79-9843712,Olenolin,Klehyn,44
41-5548000,Winonah,Skynner,44
36-7678091,Simmonds,Greensall,43
34-6222792,Hollyanne,Orcott,43
73-9322216,Hedy,Vinson,43


### Creazione della proiezione

In [3]:
NOME_GRAPH_OFFICERS = "officers"

if graph.run("call gds.graph.exists('"+NOME_GRAPH_OFFICERS+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_OFFICERS+'")')

cq = """
CALL gds.graph.project(
  '"""+NOME_GRAPH_OFFICERS+ """',    
  ['Officer', 'Crime'],   
  {INVESTIGATED_BY: {orientation: 'UNDIRECTED'} }
) """

graph.run(cq)
print("Proiezione creata correttamente!")

Proiezione creata correttamente!


### Memory estimation

In [4]:
graph.run(""" 
CALL gds.pageRank.write.estimate('""" +NOME_GRAPH_OFFICERS+"""', { writeProperty: 'pageRank' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Page Rank" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

CALL gds.degree.write.estimate('""" +NOME_GRAPH_OFFICERS+"""', { writeProperty: 'degree' })
YIELD  nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Degree Centrality" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
""" )


algoritmo,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
Page Rank,29762,57524,718808,718808,701 KiB
Degree Centrality,29762,57524,56,56,56 Bytes


### PageRank

In [5]:
cq=""" 
CALL gds.pageRank.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score AS pageRank
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, pageRank
ORDER BY pageRank DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,pageRank
48-0216838,Madelon,DeSousa,22.602140755580333
04-5664884,Cloe,Ings,21.27718767680494
58-9758255,Kania,Notti,20.835536650546473
27-2227814,Worthy,Nettles,20.39388562428801
79-9843712,Olenolin,Klehyn,19.952234598029545
86-7588000,Ricca,Miskimmon,19.952234598029545
41-5548000,Winonah,Skynner,19.952234598029545
36-7678091,Simmonds,Greensall,19.510583571771075
34-6222792,Hollyanne,Orcott,19.510583571771075
73-9322216,Hedy,Vinson,19.510583571771075


### Degree centrality

In [6]:
cq=""" 
CALL gds.degree.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS n, score
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,score
48-0216838,Madelon,DeSousa,50.0
04-5664884,Cloe,Ings,47.0
58-9758255,Kania,Notti,46.0
27-2227814,Worthy,Nettles,45.0
41-5548000,Winonah,Skynner,44.0
86-7588000,Ricca,Miskimmon,44.0
79-9843712,Olenolin,Klehyn,44.0
36-7678091,Simmonds,Greensall,43.0
34-6222792,Hollyanne,Orcott,43.0
73-9322216,Hedy,Vinson,43.0


### Closeness

In [7]:
cq=""" 
CALL gds.alpha.closeness.harmonic.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, centrality 
WITH gds.util.asNode(nodeId) AS n, centrality 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, centrality
ORDER BY centrality DESC
LIMIT 10 ;
"""
print_n_results(cq)



badge_number,nome,cognome,centrality
48-0216838,Madelon,DeSousa,0.001680051073552636
04-5664884,Cloe,Ings,0.001579248009139478
58-9758255,Kania,Notti,0.001545646987668425
27-2227814,Worthy,Nettles,0.0015120459661973724
86-7588000,Ricca,Miskimmon,0.0014784449447263197
41-5548000,Winonah,Skynner,0.0014784449447263197
79-9843712,Olenolin,Klehyn,0.0014784449447263197
36-7678091,Simmonds,Greensall,0.001444843923255267
34-6222792,Hollyanne,Orcott,0.001444843923255267
73-9322216,Hedy,Vinson,0.001444843923255267


### Hits

In [8]:
cq=""" 
CALL gds.alpha.hits.stream('"""+NOME_GRAPH_OFFICERS+ """',  {hitsIterations: 1})
YIELD nodeId, values 
WITH gds.util.asNode(nodeId) AS n, values 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, values
ORDER BY values DESC
LIMIT 10 ;
"""
print_n_results(cq)

badge_number,nome,cognome,values
48-0216838,Madelon,DeSousa,"{hub: 0.009576619680387082, auth: 0.05312791339882031}"
04-5664884,Cloe,Ings,"{hub: 0.009002022499563857, auth: 0.04994023859489109}"
58-9758255,Kania,Notti,"{hub: 0.008810490105956116, auth: 0.04887768032691469}"
27-2227814,Worthy,Nettles,"{hub: 0.008618957712348375, auth: 0.04781512205893828}"
86-7588000,Ricca,Miskimmon,"{hub: 0.008427425318740633, auth: 0.046752563790961876}"
41-5548000,Winonah,Skynner,"{hub: 0.008427425318740633, auth: 0.046752563790961876}"
79-9843712,Olenolin,Klehyn,"{hub: 0.008427425318740633, auth: 0.046752563790961876}"
36-7678091,Simmonds,Greensall,"{hub: 0.008235892925132892, auth: 0.045690005522985466}"
34-6222792,Hollyanne,Orcott,"{hub: 0.008235892925132892, auth: 0.045690005522985466}"
73-9322216,Hedy,Vinson,"{hub: 0.008235892925132892, auth: 0.045690005522985466}"


In [9]:
cq=""" 
CALL gds.betweenness.stream('"""+NOME_GRAPH_OFFICERS+ """')
YIELD nodeId, score 
WITH gds.util.asNode(nodeId) AS n, score 
RETURN n.badge_no as badge_number, n.name as nome, n.surname as cognome, score
ORDER BY score DESC
LIMIT 10 ;
"""
print_n_results(cq)


badge_number,nome,cognome,score
48-0216838,Madelon,DeSousa,1225.0
04-5664884,Cloe,Ings,1081.0
58-9758255,Kania,Notti,1035.0
27-2227814,Worthy,Nettles,990.0
41-5548000,Winonah,Skynner,946.0
86-7588000,Ricca,Miskimmon,946.0
79-9843712,Olenolin,Klehyn,946.0
36-7678091,Simmonds,Greensall,903.0
34-6222792,Hollyanne,Orcott,903.0
73-9322216,Hedy,Vinson,903.0


### Conclusioni
...

## 2. Trovare i gruppi di criminali

In [10]:
## Creo il project
NOME_GRAPH_CRIMINALS = "criminals-groups"
includi_conoscenti = False
if graph.run("call gds.graph.exists('"+NOME_GRAPH_CRIMINALS+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_CRIMINALS+'")')

cq = """ 
  CALL gds.graph.project.cypher('""" + NOME_GRAPH_CRIMINALS+ """',
  ' MATCH (p:Person) 
    WHERE 
        EXISTS ( (p)-[:PARTY_TO]->(:Crime) )                    // Persone che hanno preso parte a un crimine
        """ +  ( "" if includi_conoscenti else "//") + """ OR EXISTS ( (p)-[]-(:Person)-[:PARTY_TO]->(:Crime) )  // Conoscenti dei criminali
    RETURN id(p) as id',
  
  'MATCH (criminal)-[]-(conoscente:Person)
    RETURN DISTINCT id(conoscente) as target,
    CASE
        WHEN EXISTS ( (conoscente)-[:KNOWS_LW]-(criminal) ) THEN 10
        WHEN EXISTS ( (conoscente)-[:FAMILY_REL]-(criminal) ) THEN 8
        // WHEN EXISTS ( (conoscente)-[:KNOWS]-(criminal) ) THEN 5
        WHEN EXISTS ( (conoscente)-[:KNOWS_PHONE]-(criminal) ) THEN 4
        WHEN EXISTS ( (conoscente)-[:KNOWS_SN]-(criminal) ) THEN 3
    END AS peso, id(criminal) as source',
  {validateRelationships:false}
); 
"""
graph.run(cq)
print("Proiezione crata correttamente!")

Proiezione crata correttamente!


### Memory estimation

In [11]:
graph.run(""" 
CALL gds.louvain.write.estimate('""" + NOME_GRAPH_CRIMINALS+ """', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Louvain" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

CALL gds.wcc.write.estimate('""" + NOME_GRAPH_CRIMINALS+ """', { writeProperty: 'component' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Weakly Connected Components" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

""" )


algoritmo,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
Louvain,29,30,7153,566272,[7153 Bytes ... 553 KiB]
Weakly Connected Components,29,30,328,328,328 Bytes


### Algoritmo di Louvain

In [12]:

def print_in_columns(my_list):
    columns = 2  #if len(my_list)>10 else 3
    len_max = str(max([len(i) for i in my_list]) + 4)
    for i in range(0, len(my_list), columns):
        print("  ", end="")
        [eval('print(f"{i:'+len_max+'}", end="")') for i in my_list[i:i+columns]]
        print("")

def print_id_gruppo(query_results):
  for (communityId, gruppo) in query_results:
      print("---------- Gruppo "+str(communityId)+" ----------")
      # print(gruppo)
      print_in_columns(gruppo)
      print("")


In [13]:
cq = """ 
CALL gds.louvain.stream('""" + NOME_GRAPH_CRIMINALS + """', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 28 ----------
  Diana Murray (900-41-3309)          Jessica Kelly (311-75-6483)         
  Kathleen Peters (250-75-5238)       Phillip Williamson (337-28-4424)    
  Raymond Walker (879-22-8665)        

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Brian Morales (335-71-7747)    
  Jack Powell (249-54-6589)      

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### Modularity Optimization

In [14]:
cq = """ 
CALL gds.beta.modularityOptimization.stream('""" + NOME_GRAPH_CRIMINALS + """', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) AS node, communityId
WITH communityId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 14 ----------
  Diana Murray (900-41-3309)       Kathleen Peters (250-75-5238)    
  Raymond Walker (879-22-8665)     

---------- Gruppo 17 ----------
  Alan Ward (881-20-2396)        Brian Morales (335-71-7747)    
  Jack Powell (249-54-6589)      

---------- Gruppo 24 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 25 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      

---------- Gruppo 28 ----------
  Jessica Kelly (311-75-6483)         Phillip Williamson (337-28-4424)    



### Weakly Connected Components

In [15]:
cq = """ 
CALL gds.wcc.stream('""" + NOME_GRAPH_CRIMINALS + """', {relationshipWeightProperty: 'peso' })
  YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS node, componentId
WITH componentId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 13 ----------
  Alan Ward (881-20-2396)             Brian Morales (335-71-7747)         
  Diana Murray (900-41-3309)          Jack Powell (249-54-6589)           
  Jessica Kelly (311-75-6483)         Kathleen Peters (250-75-5238)       
  Phillip Williamson (337-28-4424)    Raymond Walker (879-22-8665)        

---------- Gruppo 19 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 20 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### Strongly Connected Components 


In [16]:
cq = """ 
CALL gds.alpha.scc.stream('""" + NOME_GRAPH_CRIMINALS + """')
  YIELD nodeId, componentId
WITH gds.util.asNode(nodeId) AS node, componentId
WITH componentId, apoc.coll.sort(collect(node.name + " " + node.surname + " (" + node.nhs_no+")")) AS gruppo
WHERE size(gruppo)>1  
RETURN *
"""
print_id_gruppo(graph.run(cq))

---------- Gruppo 13 ----------
  Alan Ward (881-20-2396)             Brian Morales (335-71-7747)         
  Diana Murray (900-41-3309)          Jack Powell (249-54-6589)           
  Jessica Kelly (311-75-6483)         Kathleen Peters (250-75-5238)       
  Phillip Williamson (337-28-4424)    Raymond Walker (879-22-8665)        

---------- Gruppo 19 ----------
  Billy Moore (846-48-9238)         Lillian Martinez (397-28-4474)    

---------- Gruppo 20 ----------
  Andrea Montgomery (351-83-4608)    Donald Robinson (873-28-7561)      



### Conclusioni
...

## 3. Shortest path tra persone con precedenti di spaccio o possesso di armi
Fare lo shortest path per cercare possibili connessioni e raggruppare in base alla posizione geografica

Prendo due persone: una coinvolta in un crimine di tipo Drugs e l'altra pregiudicata ma che non abbia mai compiuto un crimine Drugs. Queste due persone inoltre non devono conoscersi direttamente:

In [17]:
cq = """ 
MATCH (p1:Person)-->(c1:Crime {type:"Drugs"}), (p2:Person)-->(c2:Crime)
WHERE NOT EXISTS ( (p2)-->(:Crime {type:"Drugs"} )) AND NOT EXISTS ( (p1)-[]-(p2))
RETURN p1.nhs_no AS spacciatore_nhs_no, p1.name AS spacciatore_name, p2.nhs_no AS pregiudicato_nhs_no, p2.name AS pregiudicato_name
LIMIT 1
"""
display(graph.run(cq))


data = graph.run(cq).data()[0]
spacciatore_nhs_no = data["spacciatore_nhs_no"]
pregiudicato_nhs_no = data["pregiudicato_nhs_no"]

spacciatore_nhs_no,spacciatore_name,pregiudicato_nhs_no,pregiudicato_name
879-22-8665,Raymond,821-11-2735,Stephanie


Creo la proiezione:

In [18]:
NOME_GRAPH_DRUGS = "drugs-groups"
if graph.run("call gds.graph.exists('"+NOME_GRAPH_DRUGS+"')").data()[0]["exists"]:
  graph.run('call gds.graph.drop("'+NOME_GRAPH_DRUGS+'")')


cq = """ 
  CALL gds.graph.project.cypher(
    '"""+NOME_GRAPH_DRUGS+"""',
    'MATCH (p) RETURN id(p) as id',
    'MATCH (a)-[r]-(b)
     RETURN id(a) as source, id(b) as target, 
      CASE
        WHEN EXISTS ( (a)-[:CURRENT_ADDRESS]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:HAS_PHONE]-(b) ) THEN 5
        WHEN EXISTS ( (a)-[:HAS_POSTCODE]-(b) ) THEN 2
        WHEN EXISTS ( (a)-[:POSTCODE_IN_AREA]-(b) ) THEN 3
        WHEN EXISTS ( (a)-[:LOCATION_IN_AREA]-(b) ) THEN 3
        WHEN EXISTS ( (a)-[:CALLER]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:CALLED]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:OCCURRED_AT]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:INVOLVED_IN]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:PARTY_TO]-(b) ) THEN 1        
        WHEN EXISTS ( (a)-[:KNOWS_LW]-(b) ) THEN 1
        WHEN EXISTS ( (a)-[:FAMILY_REL]-(b) ) THEN 2
        WHEN EXISTS ( (a)-[:KNOWS_PHONE]-(b) ) THEN 7
        WHEN EXISTS ( (a)-[:KNOWS_SN]-(b) ) THEN 9
        ELSE 1000
      END AS peso, type(r) AS type',
    {validateRelationships:false}
);  
"""
graph.run(cq)

nodeQuery,relationshipQuery,graphName,nodeCount,relationshipCount,projectMillis
MATCH (p) RETURN id(p) as id,"MATCH (a)-[r]-(b) RETURN id(a) as source, id(b) as target, CASE WHEN EXISTS ( (a)-[:CURRENT_ADDRESS]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:HAS_PHONE]-(b) ) THEN 5 WHEN EXISTS ( (a)-[:HAS_POSTCODE]-(b) ) THEN 2 WHEN EXISTS ( (a)-[:POSTCODE_IN_AREA]-(b) ) THEN 3 WHEN EXISTS ( (a)-[:LOCATION_IN_AREA]-(b) ) THEN 3 WHEN EXISTS ( (a)-[:CALLER]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:CALLED]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:OCCURRED_AT]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:INVOLVED_IN]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:PARTY_TO]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:KNOWS_LW]-(b) ) THEN 1 WHEN EXISTS ( (a)-[:FAMILY_REL]-(b) ) THEN 2 WHEN EXISTS ( (a)-[:KNOWS_PHONE]-(b) ) THEN 7 WHEN EXISTS ( (a)-[:KNOWS_SN]-(b) ) THEN 9 ELSE 1000 END AS peso, type(r) AS type",drugs-groups,61521,211680,3334


### Memory estimation

In [19]:
print_n_results(""" 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.shortestPath.dijkstra.write.estimate('""" + NOME_GRAPH_DRUGS + """', {sourceNode: source, targetNode: target, relationshipWeightProperty: 'peso', writeRelationshipType: 'PATH'})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Dijkstra con pesi" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'})
CALL gds.allShortestPaths.delta.write.estimate('""" + NOME_GRAPH_DRUGS + """', {sourceNode: source, relationshipWeightProperty: 'peso', writeRelationshipType: 'PATH' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "All Shortest Paths con pesi" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.shortestPath.dijkstra.write.estimate('""" + NOME_GRAPH_DRUGS + """', {sourceNode: source, targetNode: target, writeRelationshipType: 'PATH'})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "Dijkstra senza pesi" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

UNION ALL

MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'})
CALL gds.allShortestPaths.delta.write.estimate('""" + NOME_GRAPH_DRUGS + """', {sourceNode: source, writeRelationshipType: 'PATH'})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
RETURN "All Shortest Paths senza pesi" as algoritmo, nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory;

""" )


algoritmo,nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
Dijkstra con pesi,61521,211680,1984768,1984768,1938 KiB
All Shortest Paths con pesi,61521,211680,1599856,4646760,[1562 KiB ... 4537 KiB]
Dijkstra senza pesi,61521,211680,1984768,1984768,1938 KiB
All Shortest Paths senza pesi,61521,211680,1599856,4646760,[1562 KiB ... 4537 KiB]


### Dijkstra con pesi

In [20]:

def print_results_shortest_path(cq):
    """ Funzione che stampa i risultati dello shortest path in modo piu' leggibile """
    data = graph.run(cq).data()[0]
    print("Costo totale:", data["totalCost"])
    print("Costo ad ogni nodo:", data["costs"])
    print("Shortest Path:")

    ordine_nodi = data["ordine_nodi"]
    from pprint import pprint 
    for nodo in ordine_nodi:
        text = ""
        tipo_nodo = str(nodo.labels)[1:]
        diz = dict(nodo)
        if tipo_nodo == "Person":
            text = diz["name"]+ " " + diz["surname"] + " (" +diz["nhs_no"]+")"
        elif tipo_nodo == "Crime":
            text = "Reato del " + diz["date"] + ". Tipo: "+diz["type"]+". Esito: "+ diz["last_outcome"] 
        elif tipo_nodo == "Location" or tipo_nodo == "Area":
            text = tipo_nodo + " = " +  str(diz)[1:-1].replace("'", "", -1)
        else:
            text = nodo
        print("\t", text)


cq = """ 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.shortestPath.dijkstra.stream( '"""+NOME_GRAPH_DRUGS+ """', {sourceNode: source, targetNode: target, relationshipWeightProperty: 'peso'})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
WITH totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS ordine_nodi, costs
RETURN totalCost, ordine_nodi, costs ;
""" 

print_results_shortest_path(cq)
    

Costo totale: 12.0
Costo ad ogni nodo: [0.0, 2.0, 3.0, 4.0, 7.0, 10.0, 11.0, 12.0]
Shortest Path:
	 Raymond Walker (879-22-8665)
	 Phillip Williamson (337-28-4424)
	 Reato del 26/08/2017. Tipo: Vehicle crime. Esito: Investigation complete; no suspect identified
	 Location = postcode: BL7 9AR, address: 46 High Meadows, latitude: 53.619804, longitude: -2.415218
	 Area = areaCode: BL7
	 Location = postcode: BL7 9YT, address: 60 Deakins Mill Way, latitude: 53.62624, longitude: -2.442588
	 Reato del 16/08/2017. Tipo: Violence and sexual offences. Esito: Unable to prosecute suspect
	 Stephanie Hughes (821-11-2735)


![Dijkstra_con_pesi](Dijkstra_con_pesi.png)

### All Shortest Path con pesi


In [21]:
cq = """ 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.allShortestPaths.delta.stream( '"""+NOME_GRAPH_DRUGS+ """', {sourceNode: source, relationshipWeightProperty: 'peso'})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
WITH targetNode, totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS ordine_nodi, costs
WHERE targetNode = id(target)
RETURN totalCost, ordine_nodi, costs; 
"""
# print(cq)
print_results_shortest_path(cq)





Costo totale: 12.0
Costo ad ogni nodo: [0.0, 2.0, 3.0, 4.0, 7.0, 10.0, 11.0, 12.0]
Shortest Path:
	 Raymond Walker (879-22-8665)
	 Kathleen Peters (250-75-5238)
	 Reato del 31/08/2017. Tipo: Vehicle crime. Esito: Investigation complete; no suspect identified
	 Location = postcode: BL7 9DW, address: 103 Threadfold Way, latitude: 53.614334, longitude: -2.426834
	 Area = areaCode: BL7
	 Location = postcode: BL7 9YT, address: 60 Deakins Mill Way, latitude: 53.62624, longitude: -2.442588
	 Reato del 16/08/2017. Tipo: Violence and sexual offences. Esito: Unable to prosecute suspect
	 Stephanie Hughes (821-11-2735)


![All_shortest_path_con_pesi](All_shortest_path_con_pesi.png)

### Dijkstra senza pesi

In [22]:
cq = """ 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.shortestPath.dijkstra.stream( '"""+NOME_GRAPH_DRUGS+ """', {sourceNode: source, targetNode: target})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
WITH totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS ordine_nodi, costs 
RETURN totalCost, ordine_nodi, costs; 
"""
# print(cq)
print_results_shortest_path(cq)


Costo totale: 7.0
Costo ad ogni nodo: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
Shortest Path:
	 Raymond Walker (879-22-8665)
	 Phillip Williamson (337-28-4424)
	 Brian Morales (335-71-7747)
	 Brenda Edwards (778-24-6852)
	 Ashley Bennett (508-10-3584)
	 Diane Bradley (338-31-6051)
	 Pamela Gibson (838-11-7607)
	 Stephanie Hughes (821-11-2735)


![Dijkstra_senza_pesi](Dijkstra_senza_pesi.png)

### All Shortest Path (senza pesi)

In [27]:
# TODO controllare che i risultati siano coerenti con le immagini!
cq = """ 
MATCH (source:Person {nhs_no: '"""+spacciatore_nhs_no+"""'}), (target:Person {nhs_no: '"""+pregiudicato_nhs_no+"""'})
CALL gds.allShortestPaths.delta.stream( '"""+NOME_GRAPH_DRUGS+ """', {sourceNode: source})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
WITH targetNode, totalCost, [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS ordine_nodi, costs
WHERE targetNode = id(target)
RETURN totalCost, ordine_nodi, costs; 
"""
print_results_shortest_path(cq)


Costo totale: 7.0
Costo ad ogni nodo: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
Shortest Path:
	 Raymond Walker (879-22-8665)
	 Kathleen Peters (250-75-5238)
	 Alan Ward (881-20-2396)
	 Philip Gardner (775-75-2532)
	 Ashley Bennett (508-10-3584)
	 Diane Bradley (338-31-6051)
	 Pamela Gibson (838-11-7607)
	 Stephanie Hughes (821-11-2735)


![All_shortest_path_senza_pesi](All_shortest_path_senza_pesi.png)

### Conclusioni
...